In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
base_path = './drive/My Drive/Colab Notebooks/Speech Denoising/LotOfData/'

base_path_train = base_path + 'tr/'
base_path_val = base_path + 'v/'
base_path_test = base_path + 'te/'

In [3]:
!pip install librosa

In [0]:
import librosa
import pickle

In [0]:
import os
import math
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [0]:
trx = []
trs = []
trn = []

trx_val = []
trs_val = []
trn_val = []
target = []

max_width = 513
max_length = 200

from time import time

st = time()
n = 0
for file in sorted(os.listdir(base_path_train)):
  # consider only .wav files starting with trx
  if file.endswith('.wav') and file.startswith('trx'):
    if n == 20:
      break
    n += 1
    
    dirty_file_path = os.path.join(base_path_train, file)

    # check if there is corresponding target/clean file
    clean_file_name = file.replace('trx', 'trs')
    clean_file_path = os.path.join(base_path_train, clean_file_name)
    if not os.path.exists(clean_file_path):
      continue
      
    noise_file_name = file.replace('trx', 'trn')
    noise_file_path = os.path.join(base_path_train, noise_file_name)
    if not os.path.exists(clean_file_path):
      continue
        
    # load both dirty, clean and noise files
    s, sr = librosa.load(dirty_file_path, sr=None)
    train_dirty = librosa.stft(s, n_fft=1024, hop_length=512).T
    train_dirty_val = np.zeros((max_length, max_width))
    train_dirty_val[:train_dirty.shape[0], :train_dirty.shape[1]] = np.abs(train_dirty)

    s, sr = librosa.load(clean_file_path, sr=None)
    train_clean = librosa.stft(s, n_fft=1024, hop_length=512).T
    train_clean_val = np.zeros((max_length, max_width))
    train_clean_val[:train_clean.shape[0], :train_clean.shape[1]] = np.abs(train_clean)
    
    s, sr = librosa.load(noise_file_path, sr=None)
    train_noise = librosa.stft(s, n_fft=1024, hop_length=512).T
    train_noise_val = np.zeros((max_length, max_width))
    train_noise_val[:train_noise.shape[0], :train_noise.shape[1]] = np.abs(train_noise)

    trx.append(train_dirty)
    trs.append(train_clean)
    trn.append(train_noise)
    
    trx_val.append(train_dirty_val)
    trs_val.append(train_clean_val)
    trn_val.append(train_noise_val)
    
    target.append(1 * (train_clean_val > train_noise_val)) # multiply by 1 to convert boolean to integer

In [49]:
trx_val = np.array(trx_val)
trs_val = np.array(trx_val)
trn_val = np.array(trn_val)
target = np.array(target)

trx_val.shape, trs_val.shape, trn_val.shape, target.shape

((20, 200, 513), (20, 200, 513), (20, 200, 513), (20, 200, 513))

In [0]:
num_samples = len(trx_val)
batch_size = 10
num_features = 513
num_hidden = 256

learning_rate = 0.001
num_epochs = 10

In [0]:
X = tf.placeholder(tf.float32, [None, max_length, num_features])
Y = tf.placeholder(tf.float32, [None, max_length, num_features])
seqlen = tf.placeholder(tf.int32, None)

In [0]:
cell = tf.nn.rnn_cell.DropoutWrapper(tf.nn.rnn_cell.LSTMCell(num_hidden))
output, state = tf.nn.dynamic_rnn(cell, X, sequence_length=seqlen, dtype=tf.float32)

In [0]:
dense_1 = tf.layers.Dense(units=513, activation=tf.nn.sigmoid)(output)

In [0]:
# calculate loss
loss = tf.losses.mean_squared_error(labels=Y, predictions=dense_1)
train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss=loss)
init = tf.global_variables_initializer()

In [0]:
sess = tf.Session()
saver = tf.train.Saver()

In [0]:
sess.run(tf.global_variables_initializer())

for epoch in range(num_epochs):
  for i in range(0, num_samples, batch_size):
    start_idx = i
    end_idx = min(i + batch_size, num_samples)
    